# Pure NumPy MLP with Parallelism Strategies

This notebook demonstrates pure NumPy implementations of Multi-Layer Perceptrons (MLPs) with various parallelism strategies:

1. **2-Layer MLP with ReLU** - Basic implementation with forward and backward passes
2. **Batch Dimension** - Support for batch processing
3. **Row-Sharded (Tensor-Parallel) MLP** - Splitting weight matrices by rows
4. **Column-Parallel Sharded MLP** - Splitting weight matrices by columns
5. **Data-Parallel MLP** - Splitting data across workers

All implementations include:
- Forward pass (FWD)
- Backward pass (BWD) with gradient computation

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

# Set random seed for reproducibility
np.random.seed(42)

print("NumPy version:", np.__version__)

## 2. Helper Functions

In [ ]:
def relu(x: np.ndarray) -> np.ndarray:
    """ReLU activation function."""
    return np.maximum(0, x)

def relu_derivative(x: np.ndarray) -> np.ndarray:
    """Derivative of ReLU activation."""
    return (x > 0).astype(float)

def mse_loss(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    """Mean Squared Error loss."""
    return np.mean((y_pred - y_true) ** 2)

def mse_loss_derivative(y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    """Derivative of MSE loss."""
    n = y_pred.size
    return 2 * (y_pred - y_true) / n

def initialize_weights(input_size: int, output_size: int) -> Tuple[np.ndarray, np.ndarray]:
    """Initialize weights and biases using He initialization."""
    W = np.random.randn(input_size, output_size) * np.sqrt(2.0 / input_size)
    b = np.zeros((1, output_size))
    return W, b

## 3. Basic 2-Layer MLP with ReLU (Forward + Backward)

### Architecture:
- Input layer: `input_size` dimensions
- Hidden layer: `hidden_size` dimensions with ReLU activation
- Output layer: `output_size` dimensions

### Forward Pass:
```
z1 = X @ W1 + b1
a1 = ReLU(z1)
z2 = a1 @ W2 + b2
y_pred = z2
```

### Backward Pass:
```
dz2 = dL/dy_pred
dW2 = a1.T @ dz2
db2 = sum(dz2, axis=0)
da1 = dz2 @ W2.T
dz1 = da1 * ReLU'(z1)
dW1 = X.T @ dz1
db1 = sum(dz1, axis=0)
```

In [ ]:
class TwoLayerMLP:
    """2-Layer MLP with ReLU activation - Pure NumPy implementation."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.W1, self.b1 = initialize_weights(input_size, hidden_size)
        self.W2, self.b2 = initialize_weights(hidden_size, output_size)
        
        # Cache for backward pass
        self.cache = {}
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward pass.
        
        Args:
            X: Input data of shape (batch_size, input_size)
        
        Returns:
            Output of shape (batch_size, output_size)
        """
        # Layer 1: Linear + ReLU
        z1 = X @ self.W1 + self.b1  # (batch_size, hidden_size)
        a1 = relu(z1)  # (batch_size, hidden_size)
        
        # Layer 2: Linear
        z2 = a1 @ self.W2 + self.b2  # (batch_size, output_size)
        
        # Cache for backward pass
        self.cache = {
            'X': X,
            'z1': z1,
            'a1': a1,
            'z2': z2
        }
        
        return z2
    
    def backward(self, dL_dy: np.ndarray) -> dict:
        """
        Backward pass.
        
        Args:
            dL_dy: Gradient of loss w.r.t. output, shape (batch_size, output_size)
        
        Returns:
            Dictionary containing gradients for all parameters
        """
        X = self.cache['X']
        z1 = self.cache['z1']
        a1 = self.cache['a1']
        
        # Gradient for Layer 2
        dz2 = dL_dy  # (batch_size, output_size)
        dW2 = a1.T @ dz2  # (hidden_size, output_size)
        db2 = np.sum(dz2, axis=0, keepdims=True)  # (1, output_size)
        
        # Gradient for Layer 1
        da1 = dz2 @ self.W2.T  # (batch_size, hidden_size)
        dz1 = da1 * relu_derivative(z1)  # (batch_size, hidden_size)
        dW1 = X.T @ dz1  # (input_size, hidden_size)
        db1 = np.sum(dz1, axis=0, keepdims=True)  # (1, hidden_size)
        
        gradients = {
            'dW1': dW1,
            'db1': db1,
            'dW2': dW2,
            'db2': db2
        }
        
        return gradients
    
    def update_parameters(self, gradients: dict, learning_rate: float):
        """Update parameters using gradient descent."""
        self.W1 -= learning_rate * gradients['dW1']
        self.b1 -= learning_rate * gradients['db1']
        self.W2 -= learning_rate * gradients['dW2']
        self.b2 -= learning_rate * gradients['db2']

### Test 2-Layer MLP with Batch Dimension

In [ ]:
# Create synthetic data
batch_size = 32
input_size = 10
hidden_size = 20
output_size = 5

X = np.random.randn(batch_size, input_size)
y_true = np.random.randn(batch_size, output_size)

# Initialize model
model = TwoLayerMLP(input_size, hidden_size, output_size)

# Forward pass
y_pred = model.forward(X)
loss = mse_loss(y_pred, y_true)

print(f"Input shape: {X.shape}")
print(f"Output shape: {y_pred.shape}")
print(f"Initial loss: {loss:.4f}")

# Backward pass
dL_dy = mse_loss_derivative(y_pred, y_true)
gradients = model.backward(dL_dy)

print("\nGradient shapes:")
for name, grad in gradients.items():
    print(f"  {name}: {grad.shape}")

# Train for a few iterations
learning_rate = 0.01
losses = []

for epoch in range(100):
    # Forward pass
    y_pred = model.forward(X)
    loss = mse_loss(y_pred, y_true)
    losses.append(loss)
    
    # Backward pass
    dL_dy = mse_loss_derivative(y_pred, y_true)
    gradients = model.backward(dL_dy)
    
    # Update parameters
    model.update_parameters(gradients, learning_rate)
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss - 2-Layer MLP')
plt.grid(True)
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")

## 4. Row-Sharded (Tensor-Parallel) MLP

In row-sharding (also known as tensor parallelism), we split the weight matrices by rows across multiple workers.

### For Layer 1 (Input → Hidden):
- Weight matrix W1: (input_size, hidden_size)
- Split W1 by rows into N shards: W1_shard_i has shape (input_size // N, hidden_size)
- Each worker processes a portion of the input

### Forward Pass:
```
Each worker i:
  X_shard_i = X[:, shard_i_indices]
  z1_partial_i = X_shard_i @ W1_shard_i
All-Reduce: z1 = sum(z1_partial_i) + b1
a1 = ReLU(z1)
```

### Backward Pass:
```
Each worker i:
  dW1_shard_i = X_shard_i.T @ dz1
  dX_shard_i = dz1 @ W1_shard_i.T
```

In [ ]:
class RowShardedMLP:
    """Row-Sharded (Tensor-Parallel) MLP - Pure NumPy implementation."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_shards: int):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_shards = num_shards
        
        assert input_size % num_shards == 0, "input_size must be divisible by num_shards"
        self.shard_size = input_size // num_shards
        
        # Initialize sharded weights for Layer 1
        self.W1_shards = []
        for i in range(num_shards):
            W_shard, _ = initialize_weights(self.shard_size, hidden_size)
            self.W1_shards.append(W_shard)
        self.b1 = np.zeros((1, hidden_size))
        
        # Layer 2 is not sharded (for simplicity)
        self.W2, self.b2 = initialize_weights(hidden_size, output_size)
        
        # Cache for backward pass
        self.cache = {}
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward pass with row-sharding.
        
        Args:
            X: Input data of shape (batch_size, input_size)
        
        Returns:
            Output of shape (batch_size, output_size)
        """
        
        # Split input by columns (each shard gets a portion of features)
        X_shards = np.split(X, self.num_shards, axis=1)
        
        # Compute partial outputs on each shard
        z1_partials = []
        for i in range(self.num_shards):
            z1_partial = X_shards[i] @ self.W1_shards[i]  # (batch_size, hidden_size)
            z1_partials.append(z1_partial)
        
        # All-Reduce: Sum partial results
        z1 = np.sum(z1_partials, axis=0) + self.b1  # (batch_size, hidden_size)
        a1 = relu(z1)  # (batch_size, hidden_size)
        
        # Layer 2 (not sharded)
        z2 = a1 @ self.W2 + self.b2  # (batch_size, output_size)
        
        # Cache for backward pass
        self.cache = {
            'X_shards': X_shards,
            'z1': z1,
            'a1': a1,
            'z2': z2
        }
        
        return z2
    
    def backward(self, dL_dy: np.ndarray) -> dict:
        """
        Backward pass with row-sharding.
        
        Args:
            dL_dy: Gradient of loss w.r.t. output, shape (batch_size, output_size)
        
        Returns:
            Dictionary containing gradients for all parameters
        """
        X_shards = self.cache['X_shards']
        z1 = self.cache['z1']
        a1 = self.cache['a1']
        
        # Gradient for Layer 2
        dz2 = dL_dy  # (batch_size, output_size)
        dW2 = a1.T @ dz2  # (hidden_size, output_size)
        db2 = np.sum(dz2, axis=0, keepdims=True)  # (1, output_size)
        
        # Gradient for Layer 1
        da1 = dz2 @ self.W2.T  # (batch_size, hidden_size)
        dz1 = da1 * relu_derivative(z1)  # (batch_size, hidden_size)
        
        # Compute gradients for each shard
        dW1_shards = []
        for i in range(self.num_shards):
            dW1_shard = X_shards[i].T @ dz1  # (shard_size, hidden_size)
            dW1_shards.append(dW1_shard)
        
        db1 = np.sum(dz1, axis=0, keepdims=True)  # (1, hidden_size)
        
        gradients = {
            'dW1_shards': dW1_shards,
            'db1': db1,
            'dW2': dW2,
            'db2': db2
        }
        
        return gradients
    
    def update_parameters(self, gradients: dict, learning_rate: float):
        """Update parameters using gradient descent."""
        for i in range(self.num_shards):
            self.W1_shards[i] -= learning_rate * gradients['dW1_shards'][i]
        self.b1 -= learning_rate * gradients['db1']
        self.W2 -= learning_rate * gradients['dW2']
        self.b2 -= learning_rate * gradients['db2']

### Test Row-Sharded MLP

In [ ]:
# Create synthetic data
batch_size = 32
input_size = 16  # Must be divisible by num_shards
hidden_size = 20
output_size = 5
num_shards = 4

X = np.random.randn(batch_size, input_size)
y_true = np.random.randn(batch_size, output_size)

# Initialize row-sharded model
row_model = RowShardedMLP(input_size, hidden_size, output_size, num_shards)

print(f"Number of shards: {num_shards}")
print(f"Shard size: {row_model.shard_size}")
print(f"\nWeight shard shapes:")
for i, W_shard in enumerate(row_model.W1_shards):
    print(f"  W1_shard_{i}: {W_shard.shape}")

# Train for a few iterations
learning_rate = 0.01
losses = []

for epoch in range(100):
    # Forward pass
    y_pred = row_model.forward(X)
    loss = mse_loss(y_pred, y_true)
    losses.append(loss)
    
    # Backward pass
    dL_dy = mse_loss_derivative(y_pred, y_true)
    gradients = row_model.backward(dL_dy)
    
    # Update parameters
    row_model.update_parameters(gradients, learning_rate)
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss - Row-Sharded MLP')
plt.grid(True)
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")

## 5. Column-Parallel Sharded MLP

In column-sharding, we split the weight matrices by columns across multiple workers.

### For Layer 1 (Input → Hidden):
- Weight matrix W1: (input_size, hidden_size)
- Split W1 by columns into N shards: W1_shard_i has shape (input_size, hidden_size // N)
- Each worker computes a portion of the hidden layer

### Forward Pass:
```
Each worker i:
  z1_shard_i = X @ W1_shard_i + b1_shard_i
  a1_shard_i = ReLU(z1_shard_i)
Concatenate: a1 = concat([a1_shard_0, ..., a1_shard_N])
```

### Backward Pass:
```
Split da1 by columns
Each worker i:
  dz1_shard_i = da1_shard_i * ReLU'(z1_shard_i)
  dW1_shard_i = X.T @ dz1_shard_i
All-Reduce: dX = sum(dz1_shard_i @ W1_shard_i.T)
```

In [ ]:
class ColumnShardedMLP:
    """Column-Parallel Sharded MLP - Pure NumPy implementation."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_shards: int):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_shards = num_shards
        
        assert hidden_size % num_shards == 0, "hidden_size must be divisible by num_shards"
        self.shard_size = hidden_size // num_shards
        
        # Initialize sharded weights for Layer 1 (column-wise)
        self.W1_shards = []
        self.b1_shards = []
        for i in range(num_shards):
            W_shard, b_shard = initialize_weights(input_size, self.shard_size)
            self.W1_shards.append(W_shard)
            self.b1_shards.append(b_shard)
        
        # Layer 2 is not sharded (for simplicity)
        self.W2, self.b2 = initialize_weights(hidden_size, output_size)
        
        # Cache for backward pass
        self.cache = {}
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward pass with column-sharding.
        
        Args:
            X: Input data of shape (batch_size, input_size)
        
        Returns:
            Output of shape (batch_size, output_size)
        """
        
        # Compute outputs on each shard (each produces a portion of hidden layer)
        z1_shards = []
        a1_shards = []
        for i in range(self.num_shards):
            z1_shard = X @ self.W1_shards[i] + self.b1_shards[i]  # (batch_size, shard_size)
            a1_shard = relu(z1_shard)  # (batch_size, shard_size)
            z1_shards.append(z1_shard)
            a1_shards.append(a1_shard)
        
        # Concatenate shard outputs
        a1 = np.concatenate(a1_shards, axis=1)  # (batch_size, hidden_size)
        
        # Layer 2 (not sharded)
        z2 = a1 @ self.W2 + self.b2  # (batch_size, output_size)
        
        # Cache for backward pass
        self.cache = {
            'X': X,
            'z1_shards': z1_shards,
            'a1_shards': a1_shards,
            'a1': a1,
            'z2': z2
        }
        
        return z2
    
    def backward(self, dL_dy: np.ndarray) -> dict:
        """
        Backward pass with column-sharding.
        
        Args:
            dL_dy: Gradient of loss w.r.t. output, shape (batch_size, output_size)
        
        Returns:
            Dictionary containing gradients for all parameters
        """
        X = self.cache['X']
        z1_shards = self.cache['z1_shards']
        a1 = self.cache['a1']
        
        # Gradient for Layer 2
        dz2 = dL_dy  # (batch_size, output_size)
        dW2 = a1.T @ dz2  # (hidden_size, output_size)
        db2 = np.sum(dz2, axis=0, keepdims=True)  # (1, output_size)
        
        # Gradient for Layer 1
        da1 = dz2 @ self.W2.T  # (batch_size, hidden_size)
        
        # Split da1 by columns (corresponding to shards)
        da1_shards = np.split(da1, self.num_shards, axis=1)
        
        # Compute gradients for each shard
        dW1_shards = []
        db1_shards = []
        for i in range(self.num_shards):
            dz1_shard = da1_shards[i] * relu_derivative(z1_shards[i])  # (batch_size, shard_size)
            dW1_shard = X.T @ dz1_shard  # (input_size, shard_size)
            db1_shard = np.sum(dz1_shard, axis=0, keepdims=True)  # (1, shard_size)
            
            dW1_shards.append(dW1_shard)
            db1_shards.append(db1_shard)
        
        gradients = {
            'dW1_shards': dW1_shards,
            'db1_shards': db1_shards,
            'dW2': dW2,
            'db2': db2
        }
        
        return gradients
    
    def update_parameters(self, gradients: dict, learning_rate: float):
        """Update parameters using gradient descent."""
        for i in range(self.num_shards):
            self.W1_shards[i] -= learning_rate * gradients['dW1_shards'][i]
            self.b1_shards[i] -= learning_rate * gradients['db1_shards'][i]
        self.W2 -= learning_rate * gradients['dW2']
        self.b2 -= learning_rate * gradients['db2']

### Test Column-Sharded MLP

In [ ]:
# Create synthetic data
batch_size = 32
input_size = 10
hidden_size = 16  # Must be divisible by num_shards
output_size = 5
num_shards = 4

X = np.random.randn(batch_size, input_size)
y_true = np.random.randn(batch_size, output_size)

# Initialize column-sharded model
col_model = ColumnShardedMLP(input_size, hidden_size, output_size, num_shards)

print(f"Number of shards: {num_shards}")
print(f"Shard size: {col_model.shard_size}")
print(f"\nWeight shard shapes:")
for i, W_shard in enumerate(col_model.W1_shards):
    print(f"  W1_shard_{i}: {W_shard.shape}")

# Train for a few iterations
learning_rate = 0.01
losses = []

for epoch in range(100):
    # Forward pass
    y_pred = col_model.forward(X)
    loss = mse_loss(y_pred, y_true)
    losses.append(loss)
    
    # Backward pass
    dL_dy = mse_loss_derivative(y_pred, y_true)
    gradients = col_model.backward(dL_dy)
    
    # Update parameters
    col_model.update_parameters(gradients, learning_rate)
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss - Column-Sharded MLP')
plt.grid(True)
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")

## 6. Data-Parallel MLP

In data parallelism, we split the input data (batch) across multiple workers. Each worker has a full copy of the model.

### Forward Pass:
```
Each worker i:
  X_batch_i = X[shard_i_indices]
  y_pred_i = forward(X_batch_i)  # Using full model
```

### Backward Pass:
```
Each worker i:
  gradients_i = backward(X_batch_i, y_batch_i)
All-Reduce: gradients = average(gradients_i)
Update model with averaged gradients
```

In [ ]:
class DataParallelMLP:
    """Data-Parallel MLP - Pure NumPy implementation."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_workers: int):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_workers = num_workers
        
        # Each worker has a copy of the full model
        # Initialize shared weights (in practice, these would be synchronized)
        self.W1, self.b1 = initialize_weights(input_size, hidden_size)
        self.W2, self.b2 = initialize_weights(hidden_size, output_size)
        
        # Cache for backward pass
        self.cache = {}
    
    def forward_worker(self, X_batch: np.ndarray, worker_id: int) -> np.ndarray:
        """
        Forward pass for a single worker.
        
        Args:
            X_batch: Input data batch for this worker
            worker_id: ID of the worker
        
        Returns:
            Output for this batch
        """
        # Layer 1: Linear + ReLU
        z1 = X_batch @ self.W1 + self.b1
        a1 = relu(z1)
        
        # Layer 2: Linear
        z2 = a1 @ self.W2 + self.b2
        
        return z2, {'X': X_batch, 'z1': z1, 'a1': a1, 'z2': z2}
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward pass with data parallelism.
        
        Args:
            X: Input data of shape (batch_size, input_size)
        
        Returns:
            Output of shape (batch_size, output_size)
        """
        
        # Split data across workers
        # Handle cases where batch_size is not evenly divisible
        worker_batch_size = batch_size // self.num_workers
        
        outputs = []
        worker_caches = []
        
        for worker_id in range(self.num_workers):
            start_idx = worker_id * worker_batch_size
            if worker_id == self.num_workers - 1:
                # Last worker handles remainder
                end_idx = batch_size
            else:
                end_idx = start_idx + worker_batch_size
            
            X_batch = X[start_idx:end_idx]
            y_batch, cache = self.forward_worker(X_batch, worker_id)
            outputs.append(y_batch)
            worker_caches.append(cache)
        
        # Concatenate outputs from all workers
        y_pred = np.concatenate(outputs, axis=0)
        
        # Cache for backward pass
        self.cache = {'worker_caches': worker_caches}
        
        return y_pred
    
    def backward_worker(self, dL_dy_batch: np.ndarray, cache: dict) -> dict:
        """
        Backward pass for a single worker.
        
        Args:
            dL_dy_batch: Gradient for this batch
            cache: Cached values from forward pass
        
        Returns:
            Gradients for this worker
        """
        X = cache['X']
        z1 = cache['z1']
        a1 = cache['a1']
        
        # Gradient for Layer 2
        dz2 = dL_dy_batch
        dW2 = a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)
        
        # Gradient for Layer 1
        da1 = dz2 @ self.W2.T
        dz1 = da1 * relu_derivative(z1)
        dW1 = X.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)
        
        return {
            'dW1': dW1,
            'db1': db1,
            'dW2': dW2,
            'db2': db2
        }
    
    def backward(self, dL_dy: np.ndarray) -> dict:
        """
        Backward pass with data parallelism.
        
        Args:
            dL_dy: Gradient of loss w.r.t. output
        
        Returns:
            Averaged gradients from all workers
        """
        worker_caches = self.cache['worker_caches']
        worker_batch_size = batch_size // self.num_workers
        
        # Compute gradients on each worker
        worker_gradients = []
        
        for worker_id in range(self.num_workers):
            start_idx = worker_id * worker_batch_size
            if worker_id == self.num_workers - 1:
                end_idx = batch_size
            else:
                end_idx = start_idx + worker_batch_size
            
            dL_dy_batch = dL_dy[start_idx:end_idx]
            gradients = self.backward_worker(dL_dy_batch, worker_caches[worker_id])
            worker_gradients.append(gradients)
        
        # All-Reduce: Average gradients across workers
        averaged_gradients = {
            'dW1': np.mean([g['dW1'] for g in worker_gradients], axis=0),
            'db1': np.mean([g['db1'] for g in worker_gradients], axis=0),
            'dW2': np.mean([g['dW2'] for g in worker_gradients], axis=0),
            'db2': np.mean([g['db2'] for g in worker_gradients], axis=0)
        }
        
        return averaged_gradients
    
    def update_parameters(self, gradients: dict, learning_rate: float):
        """Update parameters using gradient descent."""
        self.W1 -= learning_rate * gradients['dW1']
        self.b1 -= learning_rate * gradients['db1']
        self.W2 -= learning_rate * gradients['dW2']
        self.b2 -= learning_rate * gradients['db2']

### Test Data-Parallel MLP

In [ ]:
# Create synthetic data
batch_size = 32
input_size = 10
hidden_size = 20
output_size = 5
num_workers = 4

X = np.random.randn(batch_size, input_size)
y_true = np.random.randn(batch_size, output_size)

# Initialize data-parallel model
dp_model = DataParallelMLP(input_size, hidden_size, output_size, num_workers)

print(f"Number of workers: {num_workers}")
print(f"Batch size per worker: {batch_size // num_workers}")
print(f"\nModel weight shapes:")
print(f"  W1: {dp_model.W1.shape}")
print(f"  W2: {dp_model.W2.shape}")

# Train for a few iterations
learning_rate = 0.01
losses = []

for epoch in range(100):
    # Forward pass
    y_pred = dp_model.forward(X)
    loss = mse_loss(y_pred, y_true)
    losses.append(loss)
    
    # Backward pass
    dL_dy = mse_loss_derivative(y_pred, y_true)
    gradients = dp_model.backward(dL_dy)
    
    # Update parameters
    dp_model.update_parameters(gradients, learning_rate)
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss - Data-Parallel MLP')
plt.grid(True)
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")

## 7. Comparison of All Approaches

In [ ]:
# Create common synthetic data
batch_size = 32
input_size = 16  # Divisible by 4
hidden_size = 16  # Divisible by 4
output_size = 5
num_shards = 4

X = np.random.randn(batch_size, input_size)
y_true = np.random.randn(batch_size, output_size)

# Initialize all models
models = {
    'Standard MLP': TwoLayerMLP(input_size, hidden_size, output_size),
    'Row-Sharded MLP': RowShardedMLP(input_size, hidden_size, output_size, num_shards),
    'Column-Sharded MLP': ColumnShardedMLP(input_size, hidden_size, output_size, num_shards),
    'Data-Parallel MLP': DataParallelMLP(input_size, hidden_size, output_size, num_shards)
}

# Train all models
learning_rate = 0.01
num_epochs = 100
all_losses = {}

for name, model in models.items():
    print(f"Training {name}...")
    losses = []
    
    for epoch in range(num_epochs):
        # Forward pass
        y_pred = model.forward(X)
        loss = mse_loss(y_pred, y_true)
        losses.append(loss)
        
        # Backward pass
        dL_dy = mse_loss_derivative(y_pred, y_true)
        gradients = model.backward(dL_dy)
        
        # Update parameters
        model.update_parameters(gradients, learning_rate)
    
    all_losses[name] = losses
    print(f"  Final loss: {losses[-1]:.4f}")

# Plot comparison
plt.figure(figsize=(12, 6))
for name, losses in all_losses.items():
    plt.plot(losses, label=name, linewidth=2)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MSE Loss', fontsize=12)
plt.title('Training Loss Comparison - All MLP Approaches', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Summary: All approaches converge to similar loss values.")
print("This demonstrates that parallelism strategies maintain")
print("mathematical equivalence while enabling distributed computation.")
print("="*60)

## 8. Summary and Key Takeaways

### Implementation Summary:

1. **2-Layer MLP with ReLU**
   - Pure NumPy implementation with batch support
   - Forward pass: Linear → ReLU → Linear
   - Backward pass: Computes gradients using chain rule

2. **Row-Sharded (Tensor-Parallel) MLP**
   - Splits input features across workers
   - Each worker processes a subset of input dimensions
   - Requires all-reduce operation to sum partial results
   - Memory savings: Weights distributed across workers

3. **Column-Parallel Sharded MLP**
   - Splits hidden layer neurons across workers
   - Each worker computes a portion of hidden activations
   - Outputs are concatenated
   - Useful when hidden layer is very large

4. **Data-Parallel MLP**
   - Splits batch across workers
   - Each worker has full model copy
   - Gradients are averaged across workers
   - Most common parallelism strategy in practice

### Key Concepts:

- **Forward Pass**: Compute outputs from inputs
- **Backward Pass**: Compute gradients using backpropagation
- **Batch Processing**: Process multiple samples simultaneously
- **Parallelism**: Distribute computation for efficiency
- **All-Reduce**: Synchronization operation for distributed computing

### Mathematical Equivalence:
All parallelism strategies produce mathematically equivalent results to the standard MLP, assuming proper synchronization. The choice depends on:
- Model size (tensor parallelism for large models)
- Batch size (data parallelism for large batches)
- Communication bandwidth
- Hardware constraints